In [1]:
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import seaborn as sns
import sys
import sys
import re
import os.path

## data import

In [2]:
file_name =  "clean_data_2022-05-30.csv"
folder = "../clean_equity_data/"
save = "YES"

data_tmp = pd.read_csv(folder + file_name)

In [3]:
data_tmp.columns

Index(['Unnamed: 0', 'Börsdata ID', 'Company', 'Industry', 'Volume', 'P/FCF',
       'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Market Cap',
       'Volatility', 'ROE', 'FCF', 'Total Equity', 'Return 3m', 'Return 6m',
       'Return 1y', 'Country', 'List', 'Tick', 'EV/EBIT', 'Last Report',
       'Yahoo', 'Yield', 'P/E', 'P/S', 'P/B', 'Asset Growth',
       'Info - Stock price', 'Info - Report', 'Sector', 'FCFROE', 'GPA',
       'EA ret', 'EA ret std', '1 Year Volatility', '1 Year MAD',
       'EA Volatility', 'Beta', 'Res_Mom_1Y', 'Res_Mom_6m', 'Total_Mom_6m'],
      dtype='object')

In [4]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company, data):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

In [5]:
### GESTALT
data_gestalt = data_tmp.copy()
data_gestalt = data_gestalt.loc[(data_tmp['Sector'] != 'Financials')]

### Gestalt Imputation and Rank

In [6]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median, only fundamental data
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
            'GPA', 'Assets Turn', 'FCFROE', 'Asset Growth']
for i in columns: 
    data_gestalt.loc[data_gestalt[i].isna() ,i] = data_gestalt[i].median()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data_gestalt.loc[data_gestalt[i] < 0 ,i] = data_gestalt[i].max()
    
# set nan yield & vol to 0
data_gestalt.loc[data_gestalt['Yield'].isna(),'Yield'] = 0

In [7]:
RANK_WEIGHTS = {"Momentum": 1, "Value": 1, "Quality": 1}


factor=1.0/sum(RANK_WEIGHTS.values())
for k in RANK_WEIGHTS:
    RANK_WEIGHTS[k] = RANK_WEIGHTS[k]*factor
    
    
print(sum(RANK_WEIGHTS.values()))

1.0


In [8]:
####### CALCULATE RANKS ###########
for i in ['Total_Mom_6m','Return 3m', 'Return 6m', 'Return 1y', 'EA ret std', 'Res_Mom_1Y', 'Res_Mom_6m']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank(ascending = False)

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT', 'Asset Growth']:
    data_gestalt[i +' Rank'] = data_gestalt[i].rank()
    

for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'GPA', 'Assets Turn']:
    data_gestalt[i + ' Rank'] = (data_gestalt[i]).rank(ascending = False)
    
##################### Composite ranks #######################
data_gestalt['PMOM Rank'] = (data_gestalt['Total_Mom_6m Rank'] + data_gestalt['Return 1y Rank']).rank(ascending = True)
data_gestalt['IMOM Rank'] = (data_gestalt['Res_Mom_1Y Rank'] + data_gestalt['Res_Mom_6m Rank']).rank(ascending = True)
data_gestalt['Momentum Rank'] = (data_gestalt['EA ret std Rank'] + 
                                 data_gestalt['IMOM Rank'] + data_gestalt['PMOM Rank']).rank(ascending = True)


data_gestalt['Quality Rank'] = (data_gestalt['ROE Rank'] + data_gestalt['ROC Rank'] + data_gestalt['FCFROE Rank'] + 
                        data_gestalt['GPA Rank'] + data_gestalt['Assets Turn Rank']).rank()

data_gestalt['Value Rank'] = (data_gestalt['P/E Rank'] + data_gestalt['P/B Rank'] + data_gestalt['P/S Rank'] + data_gestalt['P/FCF Rank'] + 
                      data_gestalt['EV/EBIT Rank'] + data_gestalt['Asset Growth Rank']).rank()
 


    
data_gestalt['Gestalt Rank'] = ((RANK_WEIGHTS['Value']*data_gestalt['Value Rank'] 
                                 + RANK_WEIGHTS['Momentum']*data_gestalt['Momentum Rank'] +
                                RANK_WEIGHTS['Quality']* data_gestalt['Quality Rank'])).rank()



### Data Error Check

In [9]:
#data_mom.isna().sum()
data_gestalt.isna().sum()

Unnamed: 0       0
Börsdata ID      0
Company          0
Industry         0
Volume           0
                ..
IMOM Rank        0
Momentum Rank    0
Quality Rank     0
Value Rank       0
Gestalt Rank     0
Length: 68, dtype: int64

### Sorting

In [10]:
CURRENT_BUYOUTS = ['Aspire Global', 'LeoVegas']

#### Gestalt Data Frame
gestalt_tmp = data_gestalt.sort_values(by=['Gestalt Rank'])
gestalt_tmp=gestalt_tmp[ ~gestalt_tmp['Company'].isin(CURRENT_BUYOUTS)]


In [11]:
save = "YES"
if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "GESTALT_{}".format(data_date.group(0))+ ".csv"
    gestalt_tmp.to_csv(os.path.join(folder, exp_file_name))

In [12]:
compact = ['Company', 'List','Tick', 'Gestalt Rank', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'EA ret std', '1 Year Volatility']
compact_mom = ['Company', 'List','Tick','Momentum Rank','EA ret std', '1 Year Volatility']
# numbers of stocks to select for diff strategies

stocks = 30


## TRIPLE SORT ##

gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

### Print Stock Selection

In [13]:
#gestalt 
th_props = [
  ('font-size', '14px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]
# Set CSS properties for td elements in dataframe
td_props = [('font-size', '12px'), ('text-align', 'center')]
#set caption props
caption_props = [('color', 'black'),('font-size', '22px'),
        ("text-align", "center"),
        ('font-weight', 'bold')]
# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props)
]
gestalt.index = range(1, len(gestalt)+1)
gestalt_view = (gestalt.style.apply(lambda x: ['background: lightgreen' if x.name in range(1, 15+1) 
                              else '' for i in x], axis=1).set_table_styles(styles)
                 .format({'Gestalt Rank': "{:.0f}",'Quality Rank': "{:.0f}", 'Value Rank': "{:.0f}",
                          'Momentum Rank': "{:.0f}",
                          'EA ret std': "{:.1f}",'1 Year Volatility': "{:.1%}"})
                 .set_caption("Gestalt"))
gestalt_view

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
1,Arctic Paper,Small Cap,ARP,1,58,5,7,1.3,48.6%
2,Rottneros,Mid Cap,RROS,2,53,14,4,1.5,37.0%
3,Björn Borg,Small Cap,BORG,3,14,40,25,0.6,41.7%
4,SSAB B,Large Cap,SSAB B,4,54,10,18,0.7,39.3%
5,B3 Consulting,Small Cap,B3,5,7,84,1,2.1,50.7%
6,BE Group,Small Cap,BEGR,6,28,25,53,-0.6,64.9%
7,Dedicare,Small Cap,DEDI,7,3,61,44,1.0,42.0%
8,Nilörngruppen,Small Cap,NIL B,8,9,100,2,4.4,48.0%
9,Bilia,Large Cap,BILI A,9,19,15,78,2.3,33.0%
10,Prevas,Small Cap,PREV B,11,11,42,67,-0.3,47.5%


## Reversed Ranks

In [14]:
stocks = 20

## TRIPLE SORT ##
gestalt = gestalt_tmp[0:stocks]
gestalt = gestalt[compact]

neg_gestalt = gestalt_tmp.tail(stocks)[::-1][compact]

In [15]:
gestalt_tmp.sort_values(by = 'Value Rank').head(30)

,Unnamed: 0,Börsdata ID,Company,Industry,Volume,P/FCF,Assets Turn,Gross profit,Tot. Assets,ROC,...,ROC Rank,FCFROE Rank,GPA Rank,Assets Turn Rank,PMOM Rank,IMOM Rank,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
2,2,1323,AcadeMedia,Education,11.210,2.234,0.775,11542.000,18274.000,0.09078,...,176.0,13.0,28.0,139.0,223.5,255.0,299.0,72.0,1.0,96.5
333,333,1377,Serneke,Construction & Infrastructure,0.715,10.098,1.272,415.000,6946.000,0.08785,...,183.0,121.0,288.0,43.0,147.5,103.5,108.0,159.0,2.0,55.0
211,211,117,Kabe,Auto & Equipment,1.319,5.788,1.509,502.000,2277.000,0.27829,...,48.0,34.0,189.0,25.0,93.0,101.0,75.0,47.0,3.0,12.0
243,243,137,Mekonomen,Auto & Equipment,10.925,8.888,0.937,5970.000,13304.000,0.09588,...,174.0,68.0,71.0,97.0,214.5,270.0,182.0,90.0,4.0,59.0
26,26,381,Arctic Paper,Forest & Wood Products,1.080,13.428,1.428,1909.339,5839.342,0.21394,...,76.0,107.0,112.0,28.0,6.0,24.0,7.0,58.0,5.0,1.0
376,376,224,TietoEVRY,IT Consulting,1.944,6.524,0.774,24537.689,37844.262,0.15962,...,109.0,35.0,24.0,140.0,122.0,67.0,122.5,50.5,6.0,22.0
131,131,75,EnQuest,Oil & Gas -Exploration,37.842,2.007,0.290,3463.601,42215.672,0.18789,...,90.0,3.0,279.0,260.0,23.5,33.0,10.0,116.0,7.0,15.0
256,256,151,NCC B,Construction & Infrastructure,52.183,14.340,1.900,4547.000,28095.000,0.15487,...,112.0,58.0,237.0,13.0,154.0,75.5,94.0,61.0,8.0,20.0
208,208,116,JM,Homebuilding,58.299,6.090,0.866,2875.000,17457.000,0.27028,...,49.0,31.0,235.0,117.0,243.5,309.5,278.0,71.0,9.0,92.0
347,347,1632,SSAB B,Mining - Steel & Aluminum,430.544,4.116,0.870,30234.000,123870.000,0.32426,...,35.0,53.0,156.0,116.0,16.5,8.5,18.0,54.0,10.0,4.0


In [16]:
neg_gestalt

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility
303,Q-Linea,Mid Cap,QLINEA,331.0,331.0,299.0,315.0,-1.041710,0.499880
330,Sedana Medical,First North,SEDANA,329.5,281.0,317.0,327.0,-3.371080,0.682058
159,Gapwaves,First North,GAPW B,329.5,297.0,322.0,306.0,-0.461411,0.502036
92,Cell Impact,First North,CI,327.5,305.0,308.0,311.0,-0.197534,0.792551
134,Episurf Medical,Small Cap,EPIS B,327.5,313.0,307.0,304.0,-1.480082,0.598961
140,ExpreS2ion Biotech,First North,EXPRS2,325.5,318.0,305.0,290.5,0.292186,0.836996
405,Xbrane Biopharma,Mid Cap,XBRANE,325.5,328.5,321.0,264.0,0.172249,0.591317
114,Diamyd Medical,First North,DMYD B,324.0,327.0,325.0,257.5,-0.231016,0.821812
340,Smart Eye,First North,SEYE,323.0,296.0,281.5,330.0,-3.256216,0.577072
410,Zwipe,First North,ZWIPE,322.0,300.0,302.0,303.0,-0.741177,0.600497


In [17]:

if save == "YES":
    data_date  = re.search(r'\d{4}-\d{2}-\d{2}', file_name)
    folder = '../clean_equity_data'
    exp_file_name = "rank_data_{}".format(data_date.group(0))+ ".csv"
    data_tmp.to_csv(os.path.join(folder, exp_file_name))

In [18]:
company('TietoEVRY', gestalt_tmp)[compact + ['Return 3m Rank', 'Return 6m Rank',
       'Return 1y Rank', 'EA ret std Rank']]

,Company,List,Tick,Gestalt Rank,Quality Rank,Value Rank,Momentum Rank,EA ret std,1 Year Volatility,Return 3m Rank,Return 6m Rank,Return 1y Rank,EA ret std Rank
376,TietoEVRY,Large Cap,TIETOS,22.0,50.5,6.0,122.5,-0.607437,0.231319,206.0,107.0,127.5,211.0


In [19]:
gestalt_tmp[['Momentum Rank', 'Quality Rank', 'Value Rank', 'Gestalt Rank']].corr(method = 'spearman')

,Momentum Rank,Quality Rank,Value Rank,Gestalt Rank
Momentum Rank,1.000000,0.304485,0.179980,0.651456
Quality Rank,0.304485,1.000000,0.554211,0.820621
Value Rank,0.179980,0.554211,1.000000,0.768566
Gestalt Rank,0.651456,0.820621,0.768566,1.000000
